<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-Repnet-Model" data-toc-modified-id="Load-Repnet-Model-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load Repnet Model</a></span></li><li><span><a href="#Define-a-Simple-Conv3D" data-toc-modified-id="Define-a-Simple-Conv3D-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Define a Simple Conv3D</a></span></li><li><span><a href="#Repnet-Model-Part2-(Conv3D-+-BN)" data-toc-modified-id="Repnet-Model-Part2-(Conv3D-+-BN)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Repnet Model Part2 (Conv3D + BN)</a></span></li><li><span><a href="#Convert-to-Onnx" data-toc-modified-id="Convert-to-Onnx-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Convert to Onnx</a></span><ul class="toc-item"><li><span><a href="#tf2onnx:-from_keras" data-toc-modified-id="tf2onnx:-from_keras-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>tf2onnx: from_keras</a></span></li><li><span><a href="#tf2onnx:-from_function" data-toc-modified-id="tf2onnx:-from_function-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>tf2onnx: from_function</a></span></li><li><span><a href="#tf2onnx:-load-saved_models" data-toc-modified-id="tf2onnx:-load-saved_models-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>tf2onnx: load saved_models</a></span></li></ul></li><li><span><a href="#FAQs" data-toc-modified-id="FAQs-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>FAQs</a></span></li></ul></div>

In [1]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -p numpy,sklearn,pandas
%watermark -p ipywidgets,cv2,PIL,matplotlib,plotly,netron
%watermark -p torch,torchvision,torchaudio
%watermark -p tensorflow,tensorboard,tflite
%watermark -p onnx,tf2onnx,onnxruntime,tensorrt,tvm
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, Markdown, HTML, Image, Javascript
from IPython.core.magic import register_line_cell_magic, register_line_magic, register_cell_magic
display(HTML('<style>.container { width:%d%% !important; }</style>' % 90))

import sys, os, io, time, random, math
import json, base64, requests, shutil
import os.path as osp
import numpy as np
import argparse, shlex, signal

argparse.ArgumentParser.exit = lambda *arg, **kwargs: _IGNORE_

def _IMPORT(x):
    try:
        x = x.strip()
        if x.startswith('https://'):
            x = x[8:]
        if not x.endswith('.py'):
            x = x + '.py'
        if x[0] == '/':
            with open(x) as fr:
                x = fr.read()
        else:
            x = x.replace('blob/main/', '').replace('blob/master/', '')
            if x.startswith('raw.githubusercontent.com'):
                uri = 'https://' + x
                x = requests.get(uri)
                if x.status_code == 200:
                    x = x.text
            elif x.startswith('github.com'):
                uri = x.replace('github.com', 'raw.githubusercontent.com')
                mod = uri.split('/')
                for s in ['main', 'master']:
                    uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[-3:])
                    x = requests.get(uri)
                    if x.status_code == 200:
                        x = x.text
                        break
            elif x.startswith('gitee.com'):
                mod = x.split('/')
                for s in ['/raw/main/', '/raw/master/']:
                    uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[3:])
                    x = requests.get(uri)
                    if x.status_code == 200:
                        x = x.text
                        break
        exec(x, globals())
    except:
        pass

def _DIR(x, dumps=True, ret=True):
    attrs = sorted([y for y in dir(x) if not y.startswith('_')])
    result = '%s: %s' % (str(type(x))[8:-2], json.dumps(attrs) if dumps else attrs)
    if ret:
        return result
    print(result)


numpy 1.19.5
sklearn 0.0
pandas 1.1.5
ipywidgets 7.6.3
cv2 4.5.3
PIL 8.3.1
matplotlib 3.3.4
plotly 5.3.0
netron 5.1.6
torch 1.8.1+cu101
torchvision 0.9.1+cu101
torchaudio not installed
tensorflow 2.6.0
tensorboard 2.6.0
tflite not installed
onnx 1.10.1
tf2onnx 1.10.0
onnxruntime 1.8.1
tensorrt not installed
tvm not installed


In [2]:
def display_html(port, height=600):
    from IPython import display
    from html import escape as html_escape
    frame_id = 'erlangai-frame-{:08x}'.format(random.getrandbits(64))
    shell = '''
      <iframe id='%HTML_ID%' width='100%' height='%HEIGHT%' frameborder='0'>
      </iframe>
      <script>
        (function() {
          const frame = document.getElementById(%JSON_ID%);
          const url = new URL(%URL%, window.location);
          const port = %PORT%;
          if (port) {
            url.port = port;
          }
          frame.src = url;
        })();
      </script>
    '''
    replacements = [
        ('%HTML_ID%', html_escape(frame_id, quote=True)),
        ('%JSON_ID%', json.dumps(frame_id)),
        ('%HEIGHT%', '%d' % height),
        ('%PORT%', '%d' % port),
        ('%URL%', json.dumps('/')),
    ]
    for (k, v) in replacements:
        shell = shell.replace(k, v)
    display.display(display.HTML(shell))

@register_line_magic
def netron(line):
    parser = argparse.ArgumentParser(prog='netron')
    parser.add_argument('--file', '-f', type=str, required=True, help='netron model file')
    parser.add_argument('--port', '-p', type=int, default=0, help='netron server port')
    parser.add_argument('--height', type=int, default=500, help='display netron html window hight')
    import netron
    try:
        args = parser.parse_args(shlex.split(line))
        address = netron.start(args.file, address=('0.0.0.0', args.port), browse=False)
        display_html(address[1], args.height)
    except:
        pass
 

In [3]:
# !pip3 install git+https://github.com/onnx/tensorflow-onnx

In [4]:
###
### Tensorflow ###
###

import tensorflow as tf
import tf2onnx
import onnxruntime as ort

layers = tf.keras.layers
regularizers = tf.keras.regularizers

DATA_OUTPUT = '/data/nb_data/test_conv3d'
os.makedirs(DATA_OUTPUT, exist_ok=True)

## Load Repnet Model

In [5]:
_IMPORT('gitee.com/qrsforever/blog_source_codes/AI/tensorflow/misc/ResnetPeriodEstimator')

In [6]:
BATCH_SIZE = 1
IMAGE_W = 112
IMAGE_H = 112
NUM_FRAMES = 64

PATH_TO_CKPT='/data/pretrained/cv/repnet/'
SAVED_MODEL_ROOT = '/data/nb_data/test_conv3d'
MODEL_PART1_OUTFILE = f'{SAVED_MODEL_ROOT}/model_part1_outputs.npy'

In [7]:
if not os.path.exists(MODEL_PART1_OUTFILE):
    !test ! -d $PATH_TO_CKPT && mkdir -p $PATH_TO_CKPT && \
        wget -P $PATH_TO_CKPT https://storage.googleapis.com/repnet_ckpt/checkpoint && \
        wget -P $PATH_TO_CKPT https://storage.googleapis.com/repnet_ckpt/ckpt-88.data-00000-of-00002 && \
        wget -P $PATH_TO_CKPT https://storage.googleapis.com/repnet_ckpt/ckpt-88.data-00001-of-00002 && \
        wget -P $PATH_TO_CKPT https://storage.googleapis.com/repnet_ckpt/ckpt-88.index

In [8]:
model = ResnetPeriodEstimator()
model.call = tf.function(model.call)
tf.train.Checkpoint(model=model).restore(f'{PATH_TO_CKPT}/ckpt-88').expect_partial()
test_inputs = np.random.randn(BATCH_SIZE, NUM_FRAMES, IMAGE_H, IMAGE_W, 3).astype(np.float32)
test_inputs_tensor = tf.convert_to_tensor(test_inputs)
test_outputs = model(test_inputs_tensor)
test_outputs[0].shape, test_outputs[1].shape

(TensorShape([1, 64, 32]), TensorShape([1, 64, 1]))

In [9]:
class ResnetPart1(tf.keras.models.Model):
    def __init__(self, model):
        super().__init__(name='ResnetPart1')
        self.base_model = tf.keras.models.Model(
            name='base_model', inputs=model.base_model.input,
            outputs=model.base_model.get_layer('conv4_block3_out').output)
    
    def call(self, x):
        x = tf.reshape(x, (-1, IMAGE_H, IMAGE_W, 3))
        return self.base_model(x)
    
if not os.path.exists(MODEL_PART1_OUTFILE):
    model1 = ResnetPart1(model)
    model_part1_outputs = model1(test_inputs_tensor)
    np.save(MODEL_PART1_OUTFILE, model_part1_outputs)
    print(tf.reshape(model_part1_outputs, (-1,))[:3], '-'*90, model_part1_outputs.shape)
else:
    model_part1_outputs = np.load(f'{SAVED_MODEL_ROOT}/model_part1_outputs.npy')
    
model_part1_outputs.shape

(64, 7, 7, 1024)

## Define a Simple Conv3D

In [10]:
# from tf2onnx import logging, utils
# 
# logging.basicConfig(level=5)
# utils.set_debug_mode(True)
# 
class ModelCNN3D(tf.keras.models.Model):
    def __init__(self):
        super().__init__(name='ModelCNN3D')
        self.temporal_conv_layer = layers.Conv3D(
            3, 3, padding='same', name='temporal_conv_layer',
            dilation_rate=(1, 1, 1),
            kernel_regularizer=regularizers.l2(1e-6),
            kernel_initializer='he_normal'
        )
        self.temporal_bn_layers = layers.BatchNormalization(name='temporal_bn_layers')                  
        
    def call(self, x):
        x = self.temporal_conv_layer(x)
        x = self.temporal_bn_layers(x)
        x = tf.nn.relu(x)
        return x
    
testmodel = ModelCNN3D()
inputs = np.random.randn(1, 10, 28, 28, 3)
outputs = testmodel(inputs)
tf2onnx.convert.from_keras(
    testmodel,
    input_signature=(tf.TensorSpec(shape=inputs.shape, name="x"),),
    opset=11, output_path=f'{SAVED_MODEL_ROOT}/testmodel.onnx')[0];

%netron -f {SAVED_MODEL_ROOT}/testmodel.onnx --height 360

Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
Serving '/data/nb_data/test_conv3d/testmodel.onnx' at http://0.0.0.0:37718


## Repnet Model Part2 (Conv3D + BN)

In [11]:
class ResnetPart2_0(tf.keras.models.Model):
    def __init__(self, model):
        super().__init__(name='ResnetPart2_0')
        self.temporal_conv_layer = layers.Conv3D(
            512, 3, padding='same', name='temporal_conv_layer',
            dilation_rate=(3, 1, 1), weights=model.temporal_conv_layers[0].get_weights())
        self.temporal_bn_layers = layers.BatchNormalization(axis=-1, # default
            name='temporal_bn_layers', weights=model.temporal_bn_layers[0].get_weights())                  
        
    def call(self, x):
        x = tf.reshape(x, [1, -1] + x.shape.as_list()[1:])
        x = self.temporal_conv_layer(x)
        x = self.temporal_bn_layers(x)
        x = tf.nn.relu(x)
        return tf.reduce_max(x, [2, 3])

model2_0 = ResnetPart2_0(model)
model_part2_outputs = model2_0(model_part1_outputs)
tf.reshape(model_part2_outputs, (-1,))[:3], '-'*90, model_part2_outputs.shape

(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.        , 1.0429665 , 0.54142964], dtype=float32)>,
 '------------------------------------------------------------------------------------------',
 TensorShape([1, 64, 512]))

## Convert to Onnx

### tf2onnx: from_keras

In [12]:
model2_proto_0 = tf2onnx.convert.from_keras(
    model2_0,
    input_signature=(tf.TensorSpec(shape=model_part1_outputs.shape, name="x"),),
    opset=11, output_path=f'{SAVED_MODEL_ROOT}/model2_0.onnx')[0]

In [13]:
for node in model2_proto_0.graph.node:
    print(node.name)
model2_proto_0.graph.input, model2_proto_0.graph.output

ResnetPart2_0/Reshape
ResnetPart2_0/temporal_conv_layer/Conv3D__14
Conv__18
ResnetPart2_0/temporal_bn_layers/FusedBatchNormV3
ResnetPart2_0/Relu
ResnetPart2_0/Max
ResnetPart2_0/temporal_bn_layers/FusedBatchNormV3__17


([name: "x"
 type {
   tensor_type {
     elem_type: 1
     shape {
       dim {
         dim_value: 64
       }
       dim {
         dim_value: 7
       }
       dim {
         dim_value: 7
       }
       dim {
         dim_value: 1024
       }
     }
   }
 }
 ],
 [name: "output_1"
 type {
   tensor_type {
     elem_type: 1
     shape {
       dim {
         dim_value: 1
       }
       dim {
         dim_value: 64
       }
       dim {
         dim_value: 512
       }
     }
   }
 }
 ])

In [14]:
%netron -f {SAVED_MODEL_ROOT}/model2_0.onnx --height 360

Serving '/data/nb_data/test_conv3d/model2_0.onnx' at http://0.0.0.0:36869


In [15]:
sess = ort.InferenceSession(f'{SAVED_MODEL_ROOT}/model2_0.onnx')  
inputs = {sess.get_inputs()[0].name: model_part1_outputs}
outputs = [o.name for o in sess.get_outputs()]
predictions = sess.run(outputs, inputs)
predictions[-1].shape

(1, 64, 512)

### tf2onnx: from_function

In [16]:
func2_0 = tf.function(lambda x: model2_0(x))
func2_proto_0 = tf2onnx.convert.from_function(
    function=func2_0,
    input_signature=(tf.TensorSpec(model_part1_outputs.shape, model_part1_outputs.dtype),),
    opset=11, output_path=f'{SAVED_MODEL_ROOT}/func2_0.onnx')[0]

In [17]:
%netron -f {SAVED_MODEL_ROOT}/func2_0.onnx --height 360

Serving '/data/nb_data/test_conv3d/func2_0.onnx' at http://0.0.0.0:45676


### tf2onnx: load saved_models

In [18]:
model2_0.save(f'{SAVED_MODEL_ROOT}/model2_0', save_format='tf', include_optimizer=False)

INFO:tensorflow:Assets written to: /data/nb_data/test_conv3d/model2_0/assets


In [19]:
!python3 -m tf2onnx.convert --opset 11 \
    --tag serve --signature_def serving_default \
    --saved-model {SAVED_MODEL_ROOT}/model2_0 --output {SAVED_MODEL_ROOT}/model2_0/model2.onnx 

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2021-10-22 11:24:23,764 - INFO - Signatures found in model: [serving_default].
2021-10-22 11:24:23,764 - INFO - Output names: ['output_1']
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2021-10-22 11:24:25,832 - WARNING - From /usr/local/lib/python3.6/dist-packages/tf2onnx/tf_loader.py:706: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2021-10-22 11:24:27,349 - INFO - Using tensorflow=2.6.0, onnx=1.10.1, tf2onnx=1.10.0/42e800
2021-10-22 11:24:27,349 - INFO - Using opset <onnx, 11>
2021-10-22 11:24:35,148 - INFO - Computed 2 values for constant folding
2021-10-22 11:24:39,39

In [20]:
%netron -f {SAVED_MODEL_ROOT}/model2_0/model2.onnx --height 360

Serving '/data/nb_data/test_conv3d/model2_0/model2.onnx' at http://0.0.0.0:38034


## FAQs

- [Conversion of tranpose into reshape goes wrong after 3D NDHWC convolution (FusedBatchNormV3) ][1]

[1]:https://github.com/onnx/tensorflow-onnx/issues/1749